# Imports

In [1]:
import os
import pickle

import pandas as pd
import numpy as np
import seaborn as sns

from utils import CustomCleaner, get_random_base
from scipy.stats import uniform

from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import PowerTransformer, TargetEncoder, LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, fbeta_score, make_scorer
from sklearn.multiclass import OneVsOneClassifier

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from cuml.svm import SVC as cumlSVC # Using GPU acceleration for model selection, final model will be vanilla sklearn, if SVC is chosen

import warnings; warnings.filterwarnings('ignore')

# Load data

In [2]:
df = pd.read_csv(os.path.join('..', 'clean_data', 'outliers.csv'), index_col=0)
df.head()

,encounter_id,country,patient_id,race,gender,age,weight,payer_code,outpatient_visits_in_previous_year,emergency_visits_in_previous_year,...,secondary_diagnosis,additional_diagnosis,number_diagnoses,glucose_test_result,a1c_test_result,change_in_meds_during_hospitalization,prescribed_diabetes_meds,medication,readmitted_binary,readmitted_multiclass
0,533253,USA,70110,Caucasian,Female,[70-80),?,?,0,0,...,276,466,8,NaN,NaN,No,No,[],No,>30 days
2,634063,USA,80729253,Caucasian,Female,[60-70),?,?,0,0,...,135,250,6,NaN,NaN,Ch,Yes,"['glimepiride', 'insulin']",No,No
3,890610,USA,2919042,AfricanAmerican,Male,[60-70),?,MC,0,0,...,562,455,5,NaN,NaN,No,No,[],No,No
4,654194,USA,84871971,Caucasian,Female,[70-80),?,HM,1,0,...,599,428,9,NaN,NaN,No,No,[],No,>30 days
5,269878,USA,279288,Caucasian,Female,[50-60),?,?,0,0,...,250,244,3,NaN,Norm,No,No,[],No,>30 days


# Preproc

In [3]:
cleaner = CustomCleaner()
df_clean = cleaner.fit_transform(df)

In [4]:
df_clean = df_clean.drop(columns=['country', 'weight', 'glucose_test_result', 'a1c_test_result'])

In [5]:
num_cols = df_clean.select_dtypes(include=np.number).drop(columns=['encounter_id', 'patient_id']).columns.to_list()

In [6]:
cat_cols = df_clean.select_dtypes(exclude=np.number).drop(columns=['readmitted_binary', 'readmitted_multiclass']).columns

In [7]:
X = df_clean.drop(columns=['readmitted_binary', 'readmitted_multiclass', 'encounter_id', 'patient_id'])
y = df_clean.readmitted_multiclass

label = LabelEncoder()
y = label.fit_transform(y)

In [8]:
label.classes_

array(['<30 days', '>30 days', 'No'], dtype=object)

In [9]:
pd.Series(y).value_counts()

2    37320
1    24218
0     7712
dtype: int64

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)
cv = StratifiedKFold(n_splits=5)

## Pipeline

In [11]:
cat_pipe = Pipeline([
    ('encoder', TargetEncoder(target_type='continuous'))
])

num_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('transformer', PowerTransformer()),
    ('scaler', MinMaxScaler())
])

preproc_pipe = ColumnTransformer([
    ('cat', cat_pipe, cat_cols),
    ('num', num_pipe, num_cols)
], remainder='drop')

preproc_pipe

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('encoder',
                                                  TargetEncoder(target_type='continuous'))]),
                                 Index(['race', 'gender', 'payer_code', 'admission_type', 'medical_specialty',
       'discharge_disposition', 'admission_source', 'primary_diagnosis',
       'secondary_diagnosis', 'additional_diagnosis',
       'change_in_meds_during_hospitalization', 'prescribed_diabetes...
                                  'emergency_visits_in_previous_year',
                                  'inpatient_visits_in_previous_year',
                                  'average_pulse_bpm',
                                  'length_of_stay_in_hospital',
                                  'number_lab_tests', 'non_lab_procedures',
                                  'number_of_medications', 'number_diagnoses',
                                  'number_diab_meds',
                                  'number_visits_per_patient',
                                  'n_medication_per_day', 'n_lab_tests_per_day',
                                  'prop_inpatient', 'prop_outpatient',
                                  'prop_emergency'])])

In [12]:
preproc_pipe.fit(X_train, y_train)
X_train = np.float32(preproc_pipe.transform(X_train))
X_test = np.float32(preproc_pipe.transform(X_test))

# Model

In [13]:
scorer = make_scorer(fbeta_score, beta=5, average='weighted') # Weighted f1 score, giving more importance to recall than precision

## Base model

In [14]:
y_base = get_random_base(y_test)

In [15]:
print(classification_report(y_test, y_base, target_names=label.classes_))

              precision    recall  f1-score   support

    <30 days       0.11      0.11      0.11      1928
    >30 days       0.35      0.35      0.35      6055
          No       0.54      0.54      0.54      9330

    accuracy                           0.43     17313
   macro avg       0.34      0.34      0.34     17313
weighted avg       0.43      0.43      0.43     17313



## SGD

In [16]:
# OvR by default
sgd_ovr = SGDClassifier(n_jobs=-1, class_weight='balanced', max_iter=1000)

In [17]:
cross_val_score(sgd_ovr, X_train, y_train, scoring=scorer, cv=cv, n_jobs=-1).mean()

0.6586436769791918

In [18]:
# OvO
sgd_ovo = OneVsOneClassifier(SGDClassifier(n_jobs=-1, class_weight='balanced', max_iter=1000))

In [19]:
cross_val_score(sgd_ovo, X_train, y_train, scoring=scorer, cv=cv, n_jobs=-1).mean()

0.5616991138524614

## SVC

In [20]:
# OvR
svc_ovr = cumlSVC(class_weight='balanced', multiclass_strategy='ovr')

In [21]:
cross_val_score(svc_ovr, X_train, y_train, scoring=scorer, cv=cv, n_jobs=-1).mean()

0.6383964018342555

In [22]:
# OvO
svc_ovo = cumlSVC(class_weight='balanced', multiclass_strategy='ovo')

In [23]:
cross_val_score(svc_ovo, X_train, y_train, scoring=scorer, cv=cv, n_jobs=-1).mean()

0.5753314600314486

# Hypertuning

## SVC

In [24]:
rand_params = {
    'multiclass_strategy': ['ovo', 'ovr'],
    'C': uniform(0.05, 50),
    'kernel': ['poly', 'rbf', 'linear', 'sigmoid'],
    'degree': [2, 3, 4],
    'gamma': ['auto', 'scale'],
    'coef0': uniform(0, 1) 
}

In [27]:
rand_svc = RandomizedSearchCV(cumlSVC(class_weight='balanced'), rand_params, n_iter=300, scoring=scorer, cv=cv, n_jobs=-1, verbose=2)
rand_svc.fit(X_train, y_train)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
[CV] END C=15.49827932141802, coef0=0.8254886451458093, degree=4, gamma=scale, kernel=rbf, multiclass_strategy=ovo; total time=   0.6s
[CV] END C=24.65817498045837, coef0=0.48307022446842973, degree=4, gamma=auto, kernel=rbf, multiclass_strategy=ovr; total time=   0.2s
[CV] END C=40.475057925083334, coef0=0.3494283827231368, degree=4, gamma=auto, kernel=poly, multiclass_strategy=ovr; total time=   0.2s
[W] [15:02:39.541118] SVC with the linear kernel can be much faster using the specialized solver provided by LinearSVC. Consider switching to LinearSVC if tranining takes too long.
[CV] END C=7.951485784523753, coef0=0.9620082977512804, degree=4, gamma=scale, kernel=linear, multiclass_strategy=ovr; total time=   0.3s
[CV] END C=41.25460802820767, coef0=0.8518268249412365, degree=2, gamma=scale, kernel=rbf, multiclass_strategy=ovo; total time=   0.3s
[CV] END C=45.80061305993218, coef0=0.9016658891688699, degree=2, gamma=scal

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   estimator=SVC(), n_iter=300, n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f3ac8fbaf50>,
                                        'coef0': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f3ac8f9d090>,
                                        'degree': [2, 3, 4],
                                        'gamma': ['auto', 'scale'],
                                        'kernel': ['poly', 'rbf', 'linear',
                                                   'sigmoid'],
                                        'multiclass_strategy': ['ovo', 'ovr']},
                   scoring=make_scorer(fbeta_score, beta=5, average=weighted),
                   verbose=2)

In [28]:
rand_svc.best_params_

{'C': 44.02009983416125,
 'coef0': 0.7809176129012151,
 'degree': 3,
 'gamma': 'auto',
 'kernel': 'rbf',
 'multiclass_strategy': 'ovr'}

In [29]:
y_pred = rand_svc.predict(X_test)

In [30]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.24      0.31      0.27      1928
           1       0.56      0.45      0.50      6055
           2       0.75      0.81      0.78      9330

    accuracy                           0.62     17313
   macro avg       0.52      0.52      0.52     17313
weighted avg       0.63      0.62      0.62     17313



## SGD OvR

In [31]:
rand_params = {
    'loss': ['hinge', 'log_loss', 'squared_hinge', 'perceptron', 'modified_huber'],
    'penalty': ['l1', 'l2', 'elasticnet', None],
    'l1_ratio': uniform(0, 1),
    'alpha': uniform(0.00001, 1),
    'learning_rate': ['optimal', 'invscaling', 'adaptive', 'constant'],
    'eta0': uniform(0, 1),
    'epsilon': uniform(0, 1),
    'power_t': uniform(-1, 1)
}

In [32]:
rand_sgd = RandomizedSearchCV(sgd_ovr, rand_params, n_iter=300, scoring=scorer, cv=cv, n_jobs=-1, verbose=1)
rand_sgd.fit(X_train, y_train)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


/home/vsn/.pyenv/versions/3.10.6/envs/mestrado/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/vsn/.pyenv/versions/3.10.6/envs/mestrado/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/vsn/.pyenv/versions/3.10.6/envs/mestrado/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/vsn/.pyenv/versions/3.10.6/envs/mestrado/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before converg

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   estimator=SGDClassifier(class_weight='balanced', n_jobs=-1),
                   n_iter=300, n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f3ac8fdca90>,
                                        'epsilon': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f3cc...
                                        'l1_ratio': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f3ccb8d4a90>,
                                        'learning_rate': ['optimal',
                                                          'invscaling',
                                                          'adaptive',
                                                          'constant'],
                                        'loss': ['hinge', 'log_loss',
                                                 'squared_hinge', 'perceptron',
                                                 'modified_huber'],
                                        'penalty': ['l1', 'l2', 'elasticnet',
                                                    None],
                                        'power_t': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f3ccb8d6680>},
                   scoring=make_scorer(fbeta_score, beta=5, average=weighted),
                   verbose=1)

In [33]:
rand_sgd.best_params_

{'alpha': 0.8058197939273687,
 'epsilon': 0.11859171477367458,
 'eta0': 0.28303250561455784,
 'l1_ratio': 0.23253005867598175,
 'learning_rate': 'adaptive',
 'loss': 'hinge',
 'penalty': None,
 'power_t': -0.6849907138495789}

In [34]:
rand_sgd.best_score_

0.660577862850626

In [35]:
y_pred = rand_sgd.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.32      0.01      0.02      1928
           1       0.55      0.60      0.58      6055
           2       0.73      0.83      0.77      9330

    accuracy                           0.66     17313
   macro avg       0.53      0.48      0.46     17313
weighted avg       0.62      0.66      0.62     17313



## SGD OvO

In [36]:
rand_params = {
    'estimator__loss': ['hinge', 'log_loss', 'squared_hinge', 'perceptron', 'modified_huber'],
    'estimator__penalty': ['l1', 'l2', 'elasticnet', None],
    'estimator__l1_ratio': uniform(0, 1),
    'estimator__alpha': uniform(0.00001, 1),
    'estimator__learning_rate': ['optimal', 'invscaling', 'adaptive', 'constant'],
    'estimator__eta0': uniform(0, 1),
    'estimator__epsilon': uniform(0, 1),
    'estimator__power_t': uniform(-1, 1)
}

In [37]:
rand_sgd_ovo = RandomizedSearchCV(sgd_ovo, rand_params, n_iter=300, scoring=scorer, cv=cv, n_jobs=-1, verbose=1)
rand_sgd_ovo.fit(X_train, y_train)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


/home/vsn/.pyenv/versions/3.10.6/envs/mestrado/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/vsn/.pyenv/versions/3.10.6/envs/mestrado/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/vsn/.pyenv/versions/3.10.6/envs/mestrado/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/vsn/.pyenv/versions/3.10.6/envs/mestrado/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before converg

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   estimator=OneVsOneClassifier(estimator=SGDClassifier(class_weight='balanced',
                                                                        n_jobs=-1)),
                   n_iter=300, n_jobs=-1,
                   param_distributions={'estimator__alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f3ccb8d5c00>,
                                        'estimator__epsilon': <scipy.stats._dist...
                                        'estimator__learning_rate': ['optimal',
                                                                     'invscaling',
                                                                     'adaptive',
                                                                     'constant'],
                                        'estimator__loss': ['hinge', 'log_loss',
                                                            'squared_hinge',
                                                            'perceptron',
                                                            'modified_huber'],
                                        'estimator__penalty': ['l1', 'l2',
                                                               'elasticnet',
                                                               None],
                                        'estimator__power_t': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f3ccb698ac0>},
                   scoring=make_scorer(fbeta_score, beta=5, average=weighted),
                   verbose=1)

In [38]:
rand_sgd_ovo.best_params_

{'estimator__alpha': 0.307520189653321,
 'estimator__epsilon': 0.9935271890314104,
 'estimator__eta0': 0.03761823466274239,
 'estimator__l1_ratio': 0.4330466987065259,
 'estimator__learning_rate': 'optimal',
 'estimator__loss': 'modified_huber',
 'estimator__penalty': 'elasticnet',
 'estimator__power_t': -0.8886132269824396}

In [39]:
rand_sgd_ovo.best_score_

0.6306738121606362

In [40]:
y_pred = rand_sgd_ovo.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1928
           1       0.55      0.61      0.58      6055
           2       0.72      0.83      0.77      9330

    accuracy                           0.66     17313
   macro avg       0.43      0.48      0.45     17313
weighted avg       0.58      0.66      0.62     17313



# Final model and results

In [43]:
# Best params found by RandomSearchCV, using as scoring the fbeta.
# The fbeta is a modified f1 scorer, in which we can give weight to the recall agains the precision.

best_params = {'C': 44.02009983416125,
               'coef0': 0.7809176129012151,
               'degree': 3,
               'gamma': 'auto',
               'kernel': 'rbf',
               'decision_function_shape': 'ovr'}

In [44]:
model_path = os.path.join('..', 'pipelines', 'models', 'best_model.pkl')
if not os.path.exists(model_path):
    final_model = SVC(class_weight='balanced', **best_params)
    final_model.fit(X_train, y_train)
    with open(model_path, 'wb') as f:
        pickle.dump(final_model, f)
else:
    with open(model_path, 'rb') as f:
        final_model = pickle.load(f)

final_model

SVC(C=44.02009983416125, class_weight='balanced', coef0=0.7809176129012151,
    gamma='auto')

In [45]:
y_pred = final_model.predict(X_test)

In [52]:
print('BEST MODEL METRICS')
print('_' * 55)
print(classification_report(y_test, y_pred, target_names=label.classes_))

BEST MODEL METRICS
_______________________________________________________
              precision    recall  f1-score   support

    <30 days       0.23      0.47      0.31      1928
    >30 days       0.57      0.33      0.42      6055
          No       0.75      0.80      0.77      9330

    accuracy                           0.60     17313
   macro avg       0.52      0.53      0.50     17313
weighted avg       0.63      0.60      0.60     17313



In [53]:
print('DUMMY MODEL METRICS')
print('_' * 55)
print(classification_report(y_test, y_base, target_names=label.classes_))

DUMMY MODEL METRICS
_______________________________________________________
              precision    recall  f1-score   support

    <30 days       0.11      0.11      0.11      1928
    >30 days       0.35      0.35      0.35      6055
          No       0.54      0.54      0.54      9330

    accuracy                           0.43     17313
   macro avg       0.34      0.34      0.34     17313
weighted avg       0.43      0.43      0.43     17313

[W] [15:02:38.788973] SVC with the linear kernel can be much faster using the specialized solver provided by LinearSVC. Consider switching to LinearSVC if tranining takes too long.
[CV] END C=44.796137684290905, coef0=0.20361310715799952, degree=3, gamma=auto, kernel=linear, multiclass_strategy=ovr; total time=   0.7s
[CV] END C=18.35449817932975, coef0=0.7085329114514962, degree=2, gamma=auto, kernel=linear, multiclass_strategy=ovo; total time=   0.2s
[CV] END C=12.631229830946861, coef0=0.2906985188080662, degree=2, gamma=auto, kernel=